# Study script for Word2Vec models

In [1]:
import numpy as np
import pandas as pd

## Load Data

In [2]:
train = pd.read_pickle("../datastore/train.p")
train = train.drop(['_id', 'author', 'description', 'link', 'imageURL', 'providerNewsID'], axis=1)
train.head()

,category,content,provider,publishedAt,title
0,사회 > 카드뉴스,과연 진실은... \n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] \n\n[ 조선일...,chosun,2016-09-21 19:28:00,[카드뉴스] 대통령은 왜 흙을 밟지 않았을까요?
1,사회 > 카드뉴스,'세기의 연인'이라 불렸던 커플\n\n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] ...,chosun,2016-09-21 19:10:00,[카드뉴스] '브란젤리나'의 시작과 끝
2,사회 > 카드뉴스,당신의 목소리는 그 누구의 목소리보다 귀하게 쓰였습니다. 기사보기 : https:...,chosun,2016-09-21 19:03:00,[카드뉴스] 목소리를 가장 귀하게 쓴 한 성우 지망생
3,문화 > 오늘의 운세,36년생 정도를 지키면 무난한 날. 48년생 결정할 일 결정하도록. 60년생 가뭄에...,chosun,2016-09-21 18:00:00,[오늘의 운세] 9월 22일 목요일(음력 8월 22일 丁未)
4,스포츠ㆍ연예 > 스포츠 > 종합,,chosun,2016-09-21 03:00:00,[오늘의 경기] 2016년 9월 21일


## Preprocessing
1. Remove stopwords (regex, hanja)
2. POS Tagging with KoNLPy, Mecab
3. NNP to English

In [3]:
from konlpy.tag import Mecab
import hanja
import cnouns
import re

In [4]:
mecab = Mecab()

In [5]:
def text_cleaning(text):
    text = hanja.translate(text, 'substitution')
    text = re.sub('[^가-힝0-9a-zA-Z\\s]', '', text)
    text = text.replace(u"카드뉴스", '').replace(u"조선일보", '')
    return text

In [6]:
train['title'] = train['title'].apply(lambda text: text_cleaning(text))
title = [cnouns.tokenize(each[1]['title']) for each in train.iterrows()]

In [13]:
title

['대통령 흙',
 '브란 젤리 시작 끝 브란젤리',
 '목소리 성우 지망생 성우지망생',
 '오늘 운세 목요일 음력 정미 목요일음력 오늘의운세',
 '오늘 경기 오늘의경기',
 '리콜 대상 결함 차량 도로 주행 리콜대상 결함차량 도로주행',
 '사설 국민 불안 무용지물 안전 처 정부 안전 지대 사설국민 무용지물안전 안전처 안전지대 무용지물안전처',
 '사설 미 증거 수사 나선 이중 플레이 사설미 수사나선 이중플레이 미가증거',
 '사설 신동빈 회장 소환 롯데 리스크 해소 때 사설신동빈 신동빈회장 회장소환 롯데리스크 리스크해소 사설신동빈회장 신동빈회장소환 롯데리스크해소',
 '조선 칼럼 고령 저금리 덫 국민연금 조선칼럼 저금리덫',
 '만물상 전문직 공무원 만물상전문직 전문직공무원 만물상전문직공무원',
 '정민 세설신어 과 성당 과 성당 살 과성당 과성당 성당살 정민의세설신어 과성당살',
 '일자 면 전 우즈 박인비 판 기사 면전 판기사',
 '중국어 한마디 중국어한마디',
 '전문 기자 칼럼 환자 동시 수술 관행 전문기자 기자칼럼 환자동시 동시수술 수술관행 전문기자칼럼 환자동시수술 동시수술관행',
 '기고 패럴림픽 퍼스트 각오 평창 대회 준비 기고패럴림픽 패럴림픽퍼스트 평창대회 대회준비 기고패럴림픽퍼스트 퍼스트의각오 평창대회준비',
 '트렌드 돋보기 오빠 부대 트렌드돋보기 오빠부대',
 '김명환 시간 여행 노인 버스 좌석 양보 단속 대통령 자리 양보 풍습 확대 강조 시간여행 버스좌석 좌석양보 단속대통령 자리양보 양보풍습 풍습확대 확대강조 김명환의시간 버스좌석양보 대통령이자리 자리양보풍습 양보풍습확대 풍습확대강조',
 '경제 초점 산업 혁명 러다이트 운동 경제초점 산업혁명 러다이트운동 혁명이러다이트',
 '김기봉 히스토리아 쿠오바디스 마키아벨리 국가 위기 해법 히스토리아쿠오바디스 국가위기 김기봉의히스토리아 위기의해법',
 '꽃씨',
 '가슴 동시 글 짝 글짝',
 '발언 지진 보험 도입 정부 지진보험 보험도입 도입정부 지진보험도입 보험도입정부',
 '조선 대권

In [36]:
title[0].split(' ')

['대통령', '흙']

In [44]:
title_word = []
for each in title:
    title_word.append(each.split(' '))

## Training Word2Vec models
Word Embedding : Word2Vec - CBOW

In [17]:
from gensim.models import word2vec
import multiprocessing

In [41]:
model = word2vec.Word2Vec(title_word, size=100, window=5, min_count=1, workers=multiprocessing.cpu_count())
model.init_sims(replace=True)

In [55]:
model.most_similar(u'스포츠')

[('브리핑', 0.9311978220939636),
 ('김연아', 0.917351484298706),
 ('아이스하키', 0.9152438044548035),
 ('인삼', 0.9096077680587769),
 ('김재열', 0.9086613059043884),
 ('테니스', 0.8960433602333069),
 ('스포츠연구소', 0.8944023847579956),
 ('우라', 0.8936847448348999),
 ('쑨양', 0.8894104957580566),
 ('소니', 0.8859719038009644)]